In [1]:
from load_results import load_result_dataset
import pandas as pd

pn1 = 'full_fine_tuning_50epochs_edge_paper_final2'
pn2 = 'full_fine_tuning_50epochs_paper_final2'
final_data1 = load_result_dataset(pn1, pn2)
df1 = pd.DataFrame(final_data1)
df1['ft_strategy'] = 'full_fine_tuning_50epochs'

pn1 = 'full_fine_tuning_5epochs_edge_article1'
pn2 = 'full_fine_tuning_5epochs_article1'
final_data2 = load_result_dataset(pn1, pn2)
df2 = pd.DataFrame(final_data2)
df2['ft_strategy'] = 'full_fine_tuning_5epochs'

pn1 = 'linearprobe_50epochs_edge_paper_final2'
pn2 = 'linearprobe_50epochs_paper_final2'
final_data3 = load_result_dataset(pn1, pn2)
df3 = pd.DataFrame(final_data3)
df3['ft_strategy'] = 'linearprobe_50epochs'



../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_paper_final2/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_50epochs_

In [2]:
df = pd.concat([ df1,df2,df3 ], axis=0, ignore_index=True) 
# df = df[ df.model_size == 2 ]

nan_percentage = (df.isna().sum().sum() / df.size) * 100
print(f"Percentage of NaN values: {nan_percentage:.2f}%")

grouped_df = df.pivot_table(
    index=['ft_strategy', 'model_type', 'loss_function', 'backbone',],
    columns='dataset',
    # dropna=False
)

# Rename column levels
grouped_df.columns.set_names(["metric", "dataset"], inplace=True)

# Swap column levels → dataset becomes level 0, metric becomes level 1
grouped_df.columns = grouped_df.columns.swaplevel(0, 1)

# (Optional) Sort columns so that all metrics are grouped within each dataset
grouped_df = grouped_df.sort_index(axis=1, level=0)

# grouped_df.to_csv("./{}.csv".format(pn1))

grouped_df.shape

Percentage of NaN values: 14.72%


(240, 48)

In [3]:

# --- existing shortcuts ----------------------------------------------------
sum_scores  = grouped_df.xs('sum',  level=1, axis=1)     # (rows × 6 datasets)
geom_scores = grouped_df.xs('geom', level=1, axis=1)     # (rows × 6 datasets)

#--- 1. add the TOTAL aggregates you already computed ----------------------
grouped_df[('TOTAL', 'score_sum')]  = sum_scores.sum(axis=1)
grouped_df[('TOTAL', 'score_geom')] = geom_scores.sum(axis=1)

# --- 2. count NaNs across datasets -----------------------------------------
grouped_df[('TOTAL', 'nan_sum_cnt')]  = sum_scores.isna().sum(axis=1)
grouped_df[('TOTAL', 'nan_geom_cnt')] = geom_scores.isna().sum(axis=1)

# --- 3. rank as before ------------------------------------------------------
df_sorted = grouped_df.sort_values(('TOTAL', 'score_sum'), ascending=False)

df_sorted

dataset                                                                                               caltech101  \
metric                                                                                                    L1_acc   
ft_strategy               model_type          loss_function backbone                                               
full_fine_tuning_50epochs fully convolutional TRADES_v2     convnext_tiny.fb_in1k                       0.822000   
                          fully attention     TRADES_v2     vit_base_patch16_clip_224.laion2b_ft_in1k   0.776000   
                          fully convolutional TRADES_v2     convnext_tiny.fb_in22k                      0.805000   
                                                            robust_convnext_tiny                        0.586667   
                                                            edgenext_small.usi_in1k                     0.794000   
...                                                                                                          ...   
linearprobe_50epochs      fully attention     CLASSIC_AT    eva02_base_patch14_224.mim_in22k            0.121000   
                                                            deit_tiny_patch16_224.fb_in1k                    NaN   
full_fine_tuning_5epochs  hybrid              TRADES_v2     mobilevit-small                                  NaN   
                                                            coat_tiny.in1k                                   NaN   
linearprobe_50epochs      hybrid              TRADES_v2     mobilevit-small                                  NaN   

dataset                                                                                                       \
metric                                                                                                L2_acc   
ft_strategy               model_type          loss_function backbone                                           
full_fine_tuning_50epochs fully convolutional TRADES_v2     convnext_tiny.fb_in1k                      0.850   
                          fully attention     TRADES_v2     vit_base_patch16_clip_224.laion2b_ft_in1k  0.820   
                          fully convolutional TRADES_v2     convnext_tiny.fb_in22k                     0.842   
                                                            robust_convnext_tiny                       0.844   
                                                            edgenext_small.usi_in1k                    0.811   
...                                                                                                      ...   
linearprobe_50epochs      fully attention     CLASSIC_AT    eva02_base_patch14_224.mim_in22k             NaN   
                                                            deit_tiny_patch16_224.fb_in1k                NaN   
full_fine_tuning_5epochs  hybrid              TRADES_v2     mobilevit-small                              NaN   
                                                            coat_tiny.in1k                               NaN   
linearprobe_50epochs      hybrid              TRADES_v2     mobilevit-small                              NaN   

dataset                                                                                                          \
metric                                                                                                 Linf_acc   
ft_strategy               model_type          loss_function backbone                                              
full_fine_tuning_50epochs fully convolutional TRADES_v2     convnext_tiny.fb_in1k                      0.778000   
                          fully attention     TRADES_v2     vit_base_patch16_clip_224.laion2b_ft_in1k  0.774000   
                          fully convolutional TRADES_v2     convnext_tiny.fb_in22k                     0.763000   
                                                            robust_convnext_tiny                       0.899000   
 

In [4]:
# Flatten the columns
df_sorted.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_sorted.columns]

df_reset = df_sorted.reset_index()
sub_df = df_reset[["ft_strategy", "model_type", "backbone", "loss_function", "TOTAL_score_sum"]]

# Now access the columns

sub_df.to_csv("./global_dataset.csv")

In [5]:
import pandas as pd
import plotly.express as px


custom_palette = ["#E69F00", "#56B4E9", "#009E73"]  # choose a set above

# ── 1.  Rank backbones by their aggregate TOTAL_score_sum ───────────────────────
backbone_order = (
    sub_df.groupby("backbone")["TOTAL_score_sum"]
          .sum()                       # sum over the 6 settings
          .sort_values(ascending=False)
          .index                       # → ordered list of backbones
          .tolist()
)

# Make the column an ordered categorical so Plotly respects the ranking
sub_df["backbone"] = pd.Categorical(
    sub_df["backbone"],
    categories=backbone_order,
    ordered=True
)

# ── 2.  Custom colours / markers ───────────────────────────────────────────────
color_map = {
    "full_fine_tuning_50epochs": custom_palette[0],#"orange",
    "full_fine_tuning_5epochs":  custom_palette[1],#"purple",
    "linearprobe_50epochs": custom_palette[2]     #"green",
}
symbol_map = {"TRADES_v2": "star", "CLASSIC_AT": "square"}

# ── 3.  Plot ───────────────────────────────────────────────────────────────────
fig = px.scatter(
    sub_df,
    color_discrete_sequence=custom_palette,
    x= "backbone",
    y="TOTAL_score_sum",
    color="ft_strategy",
    symbol="loss_function",
    color_discrete_map=color_map,
    symbol_map=symbol_map,
    hover_data=["backbone", "ft_strategy", "loss_function", "TOTAL_score_sum"],
    height=600,
    #title="TOTAL_score_sum by Backbone"
)
fig.update_layout(
    paper_bgcolor="white",   # outside the plotting area
    plot_bgcolor="white",    # inside the plotting area

    xaxis=dict(
        showgrid=True,
        gridcolor="lightgrey",
        zeroline=False       # optional: hide the heavy axis line
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor="lightgrey",
        zeroline=False
    )
)

fig.update_layout(yaxis_title="Aggregate Accuracy Score")

fig.update_layout(showlegend=True)   # keep or turn off if it clutters
fig.show()


/var/folders/v7/3s0lms795672_f7_mh2x6bcr0000gn/T/ipykernel_71216/3108635847.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["backbone"] = pd.Categorical(


In [6]:
# ── 0.  Make sure model_type is present ─────────────────────────────────────────
# sub_df must have a column called "model_type" that assigns each backbone
# to exactly one family (e.g. "ViT", "ResNet", "ConvNeXt", …).

fig_df = sub_df.copy() 

# ── 1.  Rank backbones **within** each model_type ──────────────────────────────
order_df = (
    fig_df
      .groupby(["ft_strategy", "backbone"])["TOTAL_score_sum"]
      .sum()
      .reset_index()
      .sort_values(["model_type", "TOTAL_score_sum"],
                   ascending=[True, False])
)

# keep only the first time each backbone shows up
backbone_order = (
    order_df
      .drop_duplicates("backbone", keep="first")   # ← removes duplicates
      ["backbone"]
      .tolist()
)

# ── 2.  Tell pandas / Plotly to respect that order ─────────────────────────────
fig_df["backbone"] = pd.Categorical(
    fig_df["backbone"],
    categories=backbone_order,
    ordered=True
)

# ── 3.  (Optional but cleaner): let Plotly know the order explicitly ───────────
cat_orders = {"backbone": backbone_order}

# ── 4.  Plot ───────────────────────────────────────────────────────────────────
fig = px.scatter(
    fig_df,
    x="backbone",
    y="TOTAL_score_sum",
    color="ft_strategy",
    symbol="loss_function",
    color_discrete_map=color_map,
    symbol_map=symbol_map,
    category_orders=cat_orders,      # <‑‑ key line
    hover_data=["model_type", "backbone",
                "ft_strategy", "loss_function", "TOTAL_score_sum"],
    height=600,
)

fig.update_layout(
    paper_bgcolor="white",
    plot_bgcolor="white",
    xaxis=dict(showgrid=True, gridcolor="lightgrey", zeroline=False),
    yaxis=dict(showgrid=True, gridcolor="lightgrey", zeroline=False,
               title="Aggregate Accuracy Score"),
    showlegend=True,
)

fig.show()

KeyError: 'model_type'

In [ ]:
# fig_df[fig_df.ft_strategy == 'linearprobe_50epochs'].head(50)

order_df = (
    fig_df
      .groupby(["ft_strategy", "backbone", "loss"])["TOTAL_score_sum"]
      .sum()
      .reset_index()
      .sort_values([ "TOTAL_score_sum"],
                   ascending=[ False])
)

order_df

KeyError: 'loss'

In [ ]:
import pandas as pd
from scipy.stats import spearmanr, pearsonr

# ── 1.  Reshape: one row per backbone × loss_function, one column per project
pivot_df = sub_df.pivot_table(
    index=["backbone", "loss_function"],
    columns="ft_strategy",
    values="TOTAL_score_sum"
).reset_index()

# ── 2.  Prepare correlation results ───────────────────────────────────────────
project_pairs = [
    ("linearprobe_50epochs",  "full_fine_tuning_50epochs"),
    ("linearprobe_50epochs",  "full_fine_tuning_5epochs"),
    ("full_fine_tuning_5epochs", "full_fine_tuning_50epochs"),
]

correlation_results = []

for p1, p2 in project_pairs:
    if p1 in pivot_df.columns and p2 in pivot_df.columns:
        valid_rows = pivot_df[[p1, p2]].dropna()
        if len(valid_rows) >= 2:           # need ≥2 points for correlation
            pearson_corr, pearson_p   = pearsonr(valid_rows[p1], valid_rows[p2])
            spearman_corr, spearman_p = spearmanr(valid_rows[p1], valid_rows[p2])
            correlation_results.append({
                "comparison":  f"{p1}  vs  {p2}",
                "pearson":     pearson_corr,
                "pearson_p":   pearson_p,
                "spearman":    spearman_corr,
                "spearman_p":  spearman_p
            })

overall_corr_df = pd.DataFrame(correlation_results)
overall_corr_df


,comparison,pearson,pearson_p,spearman,spearman_p
0,linearprobe_50epochs vs full_fine_tuning_50e...,0.315203,4.401337e-03,0.441220,4.193422e-05
1,linearprobe_50epochs vs full_fine_tuning_5ep...,0.466238,1.308143e-05,0.524301,5.988333e-07
2,full_fine_tuning_5epochs vs full_fine_tuning...,0.583148,1.378443e-08,0.666058,1.561398e-11


In [ ]:
# ── Correlations by loss_function subgroup ────────────────────────────────────
loss_corrs = []
for loss in pivot_df["loss_function"].unique():
    subset = pivot_df[pivot_df["loss_function"] == loss]

    for p1, p2 in project_pairs:
        if p1 in subset.columns and p2 in subset.columns:
            valid_rows = subset[[p1, p2]].dropna()

            if len(valid_rows) >= 2:          # need ≥2 points for correlation
                pearson_corr, pearson_p   = pearsonr(valid_rows[p1], valid_rows[p2])
                spearman_corr, spearman_p = spearmanr(valid_rows[p1], valid_rows[p2])

                loss_corrs.append({
                    "loss_function": loss,
                    "comparison":   f"{p1}  vs  {p2}",
                    "pearson":      pearson_corr,
                    "pearson_p":    pearson_p,
                    "spearman":     spearman_corr,
                    "spearman_p":   spearman_p
                })

loss_corr_df = pd.DataFrame(loss_corrs)
loss_corr_df


,loss_function,comparison,pearson,pearson_p,spearman,spearman_p
0,CLASSIC_AT,linearprobe_50epochs vs full_fine_tuning_50e...,0.402875,0.009953,0.505760,0.000871
1,CLASSIC_AT,linearprobe_50epochs vs full_fine_tuning_5ep...,0.351841,0.025987,0.494021,0.001196
2,CLASSIC_AT,full_fine_tuning_5epochs vs full_fine_tuning...,0.473508,0.002028,0.557714,0.000185
3,TRADES_v2,linearprobe_50epochs vs full_fine_tuning_50e...,0.247856,0.123063,0.375547,0.016941
4,TRADES_v2,linearprobe_50epochs vs full_fine_tuning_5ep...,0.517340,0.000630,0.519077,0.000599
5,TRADES_v2,full_fine_tuning_5epochs vs full_fine_tuning...,0.612798,0.000026,0.640890,0.000008
